## Introduction to Data Science Assignment 3
__Parham Javan 810800008<br>__
__Yaser Azad 810800003__


## Install neccessary libraries

In [2]:
import importlib.util

# Check if PySpark is installed
if importlib.util.find_spec("pyspark") is None:
    # Install PySpark
    !pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month
from pyspark.sql.functions import mean, stddev
from pyspark.sql.functions import to_date
from pyspark.sql.functions import min, max, median, avg, format_number
from pyspark.sql.functions import col, when
from pyspark.sql.functions import split, explode


For better visualization of dataframes in jupyter notebooks, since they are rendered to html

In [3]:
from IPython.display import display, HTML
display(HTML('<style>pre { white-space: pre !important; }</style>'))


## Warm-Up

Step 1: Read the CSV file


In [4]:

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Stocks Analysis") \
    .getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv("stocks.csv", header=True, inferSchema=True)

print("CSV file reading done!")

CSV file reading done!


---
<br>
Step 2: Find out about the schema of data

In [5]:
print("The schema of data:\n")
df.printSchema()
print("number of dataframe rows:", df.count())


The schema of data:

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

number of dataframe rows: 1762


---
<br>
Step 3: Select records with closing price less than 500

In [6]:
less_than_500_df = df.filter(df['Close'] < 500).select('Open', 'Close', 'Volume') #filter based on gien condition
print("Records with closing price less than 500:\n")
less_than_500_df.show()
print("number of records:", less_than_500_df.count())

Records with closing price less than 500:

+------------------+------------------+---------+
|              Open|             Close|   Volume|
+------------------+------------------+---------+
|        213.429998|        214.009998|123432400|
|        214.599998|        214.379993|150476200|
|        214.379993|        210.969995|138040000|
|            211.75|            210.58|119282800|
|        210.299994|211.98000499999998|111902700|
|212.79999700000002|210.11000299999998|115557400|
|209.18999499999998|        207.720001|148614900|
|        207.870005|        210.650002|151473000|
|210.11000299999998|            209.43|108223500|
|210.92999500000002|            205.93|148516900|
|        208.330002|        215.039995|182501900|
|        214.910006|            211.73|153038200|
|        212.079994|        208.069996|152038600|
|206.78000600000001|            197.75|220441900|
|202.51000200000001|        203.070002|266424900|
|205.95000100000001|        205.940001|466777500|
|      

---
<br>
Step 4: Find out records with opening price more than 200 and closing price less than 200


In [7]:
open_more_than_200_close_less_than_200_df = df.filter((df['Open'] > 200) & (df['Close'] < 200))
print("Records with opening price more than 200 and closing price less than 200:\n")
open_more_than_200_close_less_than_200_df.show()
print("number of records:", open_more_than_200_close_less_than_200_df.count())

Records with opening price more than 200 and closing price less than 200:

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+

number of records: 3


---
<br>
Step 5: Extract the year from the date and save it in a new column

In [8]:
df_with_year = df.withColumn('Year', year(df['Date']))
print("The dataframe with the new row \"Year\":\n")
df_with_year.show()

The dataframe with the new row "Year":

+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700| 

---
<br>
Step 6: For each year, show the minimum volumes traded

In [9]:
min_volume_by_year = df_with_year.groupBy('Year').min('Volume').withColumnRenamed('min(Volume)', 'minVolume')
print("Minimum volumes traded for each year:\n")
min_volume_by_year.show()

Minimum volumes traded for each year:

+----+---------+
|Year|minVolume|
+----+---------+
|2015| 13046400|
|2013| 41888700|
|2014| 14479600|
|2012| 43938300|
|2016| 11475900|
|2010| 39373600|
|2011| 44915500|
+----+---------+



---
<br>
Step 7: For each year and month, show the highest low price

In [10]:
df_with_year_month = df.withColumn('Year', year(df['Date'])).withColumn('Month', month(df['Date']))
max_low_price_by_year_month = df_with_year_month.groupBy('Year', 'Month').agg(max('Low').alias('maxLow'))
print("Highest \"Low Price\" for each year and month:\n")
max_low_price_by_year_month.show()
print("\nnumber of rows:", max_low_price_by_year_month.count())

Highest "Low Price" for each year and month:

+----+-----+------------------+
|Year|Month|            maxLow|
+----+-----+------------------+
|2012|   10|        665.550026|
|2010|    7|        260.300003|
|2010|   12|        325.099991|
|2015|    2|        131.169998|
|2014|    4|        589.799988|
|2015|   12|        117.809998|
|2016|    7|            103.68|
|2016|   11|        111.400002|
|2012|    8| 673.5400089999999|
|2013|    2|473.24997699999994|
|2012|    4| 626.0000150000001|
|2012|   12|        585.500023|
|2014|   10|        107.209999|
|2016|    5|             99.25|
|2014|   12|        115.290001|
|2013|    9|        503.479988|
|2013|   10|        525.110016|
|2014|    5|        628.900002|
|2016|    2|         96.650002|
|2013|   12| 566.4100269999999|
+----+-----+------------------+
only showing top 20 rows


number of rows: 84


---
<br>
Step 8: Calculate mean and standard deviation of high price over the whole data frame


In [11]:
mean_high_price = df.select(mean('High')).collect()[0][0]
stddev_high_price = df.select(stddev('High')).collect()[0][0]
print("Mean High Price:", round(mean_high_price, 2))
print("Standard Deviation of High Price:", round(stddev_high_price, 2))

# Stop the SparkSession
spark.stop()

Mean High Price: 315.91
Standard Deviation of High Price: 186.9
